<a href="https://colab.research.google.com/github/roarkai/pytorch_notes/blob/master/%E2%80%9CAutograd_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autograd

## I. Autograd记录运算历史的方式
1. Autograd是一个自动微分系统。从概念上看,在执行forward pass中的函数运算的同时，autograd会记录一个有向无环图(DAG),这个图上记录了所有执行过的operation。这个图上,leaf是input tensor,root是output tensor。在执行backward pass时，通过从root到leaf的方向tracing此图,就可以用链式法则自动计算梯度。

2. **从具体的执行来看，autograd用Function objects graph来表达上述DAG。**

  (1) tensor使用的function是**torch.autograd.Function**的实例。这些function class中都定义了forward和backward函数。用他们对tensor做运算时，会自动创建计算图。

  (2) 可以apply()Function objects graph来计算评估图的结果。

  (3) 计算Forward pass时,autograd在执行对应的function的同时，还会构建一个graph来表示这些将要计算梯度的function.每个torch.Tensor的.grad_fn属性都是进入此图的入口。完成Forward pass后,就可以在反向传播中evaluate the graph以计算梯度。

3. **每个迭代都会从零开始重新创建图。** 这种设计是为了让forward pass中的运算过程可以使用任意Python control flow statements。因为一旦有了control flow statements，每次迭代中图的形状和大小就可能发生变化。每次迭代都重新创建图的好处是，在启动训练之前,不必编码所有可能的path。实际运行了哪些函数，最后就求哪些函数对应的梯度。

In [ ]:
import torch

x = torch.ones(5)   # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)  # 要计算梯度
b = torch.randn(3, requires_grad=True)     # 要计算梯度
z = x @ w + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

![Screenshot%202023-10-09%20at%2015.26.00.png](attachment:Screenshot%202023-10-09%20at%2015.26.00.png)

In [ ]:
# tensor的grad_fn属性中存放了ref to the BP function
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x7f597c1a00a0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7f597c1a0c40>


### 计算梯度
1. 只有计算图中leaf nodes(requires_grad设为True时)才有grad properties，其他nodes都没有。
2. 一个计算图默认只能做一次BP。这是pytorch基于性能考虑设置的规则。如果要多次BP，要在backward call中设置参数retain_graph=True。

In [ ]:
loss.backward()
print(w.grad)
print(b.grad)
print(x.grad)

tensor([[0.0012, 0.2839, 0.0744],
        [0.0012, 0.2839, 0.0744],
        [0.0012, 0.2839, 0.0744],
        [0.0012, 0.2839, 0.0744],
        [0.0012, 0.2839, 0.0744]])
tensor([0.0012, 0.2839, 0.0744])
None


### backward所需的数据如何传递
1. 自定义nn.autograd.Function时，在forward method中用save_for_backward()来存储backward中会用到的tensor data。这些数据在backward method中可以用saved_tensors() method提取。pytorch中已经定义好的函数也是通过这种方式向backward method传递data的。

2. 在debug需要的时候，可以查看grad_fn对应的saved tensors。这些tensor以grad_fn的attribute形式存在，attribute name以'_saved'开头。
3. pytorch在saving时，会将function中的saved tensor打包，在读取他们的时候再。unpack。但在读取他们的时候可能会因为要防止reference cycles，将tensor unpack成另一个tensor。此时读取出来的tensor和原来的tensor很可能不是同一个tensor。但是他们有共同的data storage。2.1版本中，只有tensor是其对应的grad_fn的output，才会这样。但这个implementation dettails在未来可能会变化。
4. 可以通过hooks for saved tensors来控制pytorch如何packing/unpacking saved tensors.

In [ ]:
# save input of function
x = torch.randn(5, requires_grad=True)
y = x.pow(2)
print(x.equal(y.grad_fn._saved_self)) #
print(x is y.grad_fn._saved_self)     #

True
True


In [ ]:
# save output of function
x = torch.randn(5, requires_grad=True)
y = x.exp()
print(x.equal(y.grad_fn._saved_result)) #
print(x is y.grad_fn._saved_result)     #

False
False


## locally disabling gradient tracking/computation
默认所有requires_grad=True的tensor都会被track computational history。但有的时候，不需要计算梯度，为了提高运算效率，可以disable gradient computation。

· **不需要做gradient computation的典型场景：** \
  (1) model已经train好了，只做inference，不需要好用资源track history\
  (2) frozen参数来做finetune

· **diable gradient tracking的两类方法：** \
  (1) disable entire block of code用context manager,比如：no-grad mode和inference mode\
  (2) 在更精细的范围内exclusion of sub-gragphs from gradient computation可以通过设置tensor的requires_grad值。

### 方法1：setting requires_grad
1. tensor的requires_grad可以用来控制forward和backward中的gradient computation是否发生。requires_grad的默认是false，但wrapped in nn.Parameter的tensor的默认值是True。
(1)Forward pass中，只有当operation中有tensor的requires_grad=True的时候才会被record到backward graph上。
(2)Backward pass: 只有requires_grad=True的leaf tensor才会计算gradients并累积到tensor.grad上。所以，尽管每个tensor都设定了requires_grad的值，实际上只有leaf tensor的这个设定是有意义的。
2. leaf tensor和non-leaf tensor的区别：
(1)leaf: 没有grad_fn; 没有backward graph与之对应；requires_grad的值要设定，要么手动设定，要么nn.Parameter中自动设定。
(2)non-leaf:有grad_fn; 有backward graph与之对应，因此他们的gradient只是作为intermediate result来计算requires_grad=True的leaf tensor的gradient；也因此，所有non-leaf tensor都自动设定了requires_grad=True，不需要手动设定。

### 方法2：setting grad mode
· pytorch中有3种grad model，他们决定了autograd处理gradient的方式：默认的是grad mode，此外还有no-grad mode和inference mode。他们都可以通过context manager和decorator来设置。
1. **<font color=red>grad mode</font>:** 只有在该mode下，requires_grad的配置才能起作用，autograd才会record operations on the tensors。在另外两种mode下，requires_grad都会被overriden to be False。

2. **<font color=red>no-grad mode</font>:** 让autograd暂时不记录operations（不将operations记录到backward graph上），退出该mode回到grad mode后再根据requires_grad的值是True还是False决定是否记录。\
  **· <font color=orange>适用场景：</font>** 只是暂时中断tensor operation的autograd，但tensor在当前operation中的运算结果还会被用于grad mode下的operation。\
  **· <font color=lightgreen>例子1, optimizer: </font>** 此时，weights' update不需要被autograd记录下来，但更新后的weights的值要用到下一次迭代中。而参与下一次迭代的forward运算时，这些weights是在grad mode下作计算的。\
  **· <font color=lightgreen>例子2, torch.nn.init</font>** 初始化operation是不需要计算梯度的。但是初始化完成后的training过程中，这些weights执行forward运算都是在grad mode中。\

3. **<font color=red>inference mode</font>:** 不是暂停让autograd记录tensor参与的operation，而是完全关闭autograd对operation的tracking。优点是tensor的运算速度会比no-grad mode更快。代价是inference mode下创建的tensor即使退出inference mode后，autograd也无法tracking它们参与的operations。\
  **· <font color=orange>适用场景：</font>** tensor参与的运算不需要记录到backward graph，并且这些运算中创建的tensor后续也不会用于其他需要被autograd记录的运算中去。\
  **· <font color=lightgreen>例子1, data processing: </font>** 不是模型的一部分，不做backward。\
  **· <font color=lightgreen>例子2, model evaluation：</font>** inference不做backward\

4. **<font color=red>evaluation mode</font>:** 这个不是disable autograd的机制，但容易和上述机制混淆。\
从功能上看，module.eval()和前面的no-grad和inference mode完全是不同的东西。理论上，model.eval()的作用取决于model中使用的modules，以及他们是否定义了trainning specific behavior。\
比如：model中用了torch.nn.Dropout或者torch.nn.Batchnorm，他们的特点是在trainning和validation/test时执行的operation不一样，这种情况下，就要手动在trainning和val/test阶段分别调用model.eval(), model.train()。
tensor的detach method。\
· <font color=red>建议：不论有没有定义trainning specific behavior，都在trainning和val/test阶段分别调用model.eval(), model.train()。因为，model中使用的module即使在现在没有trainning specific behavior，未来可能会发生改变。</font>

In [ ]:
## 不区分trainning和val/test (伪码)

# for t in range(1000):
#   y_pred_train = model(x_train)
#   loss = loss(y_pred_train, y_train)
#   loss.backward()
#   opt.step()
#   opt.zero_grad()

#   with torch.no_grad():
#     y_pred_val = model(x_val)
#     loss = loss(y_pred_val, y_val)
#     val_loss = sum(loss) / len(y_val)

In [ ]:
## 区分trainning和val/test (伪码)

# for t in range(1000):
#   model.train()
#   y_pred_train = model(x_train)
#   loss = loss(y_pred_train, y_train)
#   loss.backward()
#   opt.step()
#   opt.zero_grad()

#   model.eval()
#   with torch.no_grad():
#     y_pred_val = model(x_val)
#     loss = loss(y_pred_val, y_val)
#     val_loss = sum(loss) / len(y_val)

In [ ]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


In [ ]:
z = torch.matmul(x, w)+b
print(z.requires_grad)
z_det = z.detach()
print(z_det.requires_grad)

True
False


## directed acyclic graph(DAG)
1. DAG由Function对象组成
2. 叶节点是input tensor，根节点是output tensor，通过从root到leaves遍历graph，可以用chainrule自动计算gradients
3. forward pass中，autograd会同时做两件事：\
(1)执行forward operation，计算tensor\
(2)将operation的gradient function存到DAG中
4. 当.backward()被调用的时候，就会做backward pass。此时autograd会做3件事：
(1)用每个.grad_fn计算梯度\
(2)将梯度累积到各自对应tensor的.grad属性中\
(3)使用chain rule，向leaf tensor传梯度